In [8]:
import spotipy
import pandas as pd
import numpy as np
import spotipy
import hashlib
from datetime import date
from datetime import timedelta
from datetime import datetime
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re
import math
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from fuzzywuzzy import fuzz
tqdm_notebook.pandas()

load_dotenv('../secrets/.env', override=True)

scope = ["playlist-modify-private","playlist-read-private","user-library-modify","user-library-read","user-top-read"]

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

my_id = spotify.current_user()['id']

%run ../src/useful_functions.ipynb
%run ../src/spotify_scraping.ipynb
%run ../src/spotify_scraping_playlists.ipynb

In [9]:
tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()
playlists = get_spotify_playlists()
playlist_tracks = get_spotify_playlist_tracks()
library = get_spotify_library()
top_tracks = get_spotify_top_tracks()

In [10]:
get_library(max=20000, without_duplicates=False)
#library = library.drop_duplicates(subset='Track ID', keep='last')
#temp = library.groupby(['Track ID'])['Track ID'].count()
#temp = pd.DataFrame(temp[temp > 1].index).merge(tracks, how = 'left', on = 'Track ID')

10900it [1:54:32,  1.59it/s]


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [7]:
offset = 0
total = 1000
temp = pd.DataFrame()

while offset<total:
    temp_playlists = spotify.current_user_playlists(limit=50, offset=offset)
    temp = pd.concat([temp,pd.DataFrame.from_dict({'Playlist ID': [x['id'] for x in temp_playlists['items']],
                                                   'Playlist Name': [x['name'] for x in temp_playlists['items']],
                                                   'Playlist Description': [x['description'] for x in temp_playlists['items']]})])
    if offset == 0:
        total = temp_playlists['total']
    offset += 50

del(offset, total, temp_playlists)

KeyboardInterrupt: 

In [5]:
playlists_to_run = list(temp[[((x == '**') or (x[0:3] == '** ')) for x in temp['Playlist Description']]]['Playlist ID']) 
for playlist_id in tqdm(playlists_to_run):
    get_playlist(playlist_id, with_tracks = True)
    
playlists_to_run = list(temp[[((x == '*****') or (x[0:6] == '***** ')) for x in temp['Playlist Description']]]['Playlist ID']) 
for playlist_id in tqdm(playlists_to_run):
    get_playlist(playlist_id, with_tracks = True)

10900it [04:02, 44.90it/s]
100%|██████████| 4/4 [01:33<00:00, 23.38s/it]


In [6]:
playlists_to_run = list(pd.merge(pd.DataFrame(["New Music Friday UK",
                            "🆕🎧 New Alternative 40 Chart [Official] | Updated Weekly",
                            "Best tracks from 2023 albums ",
                            "Hot Tracks - 2023's best new singles",
                            "The Times Essential Tracks",
                            "The Needle Drop Weekly Track Roundup -  Fantano’s Best, Mehst and Worst New Tracks",
                            "Discover Weekly",
                            "Release Radar",
                            "R1 Weekly Playlist",
                            "BBC Radio 6 Music 2023",
                            #"Indie Show with Jack Saunders",
                            #"Radio 1's Future Sounds with Clara Amfo",
                            #"Radio 1's Future Artists with Jack Saunders",
                            #"Radio 1's Future Alternative",
                            #"Radio 1's Chillest Show",
                            #"BBC Introducing on Radio 1",
                            #"Radio 1's Power Down Playlist with Sian Eleri",
                            #"New Music Fix",
                            #"Steve Lamacq",
                            "Top Tracks",
                            "On Repeat",
                            "Repeat Rewind"
                            ], columns = ["Playlist Name"]),temp, on = 'Playlist Name', how = 'left')['Playlist ID'])

In [7]:
for playlist_id in tqdm(playlists_to_run):
    get_playlist(playlist_id, with_tracks = True)

get_top_tracks()

del(temp, playlists_to_run)

100%|██████████| 13/13 [00:27<00:00,  2.10s/it]


In [8]:
mapping_df = tracks.merge(library, on='Track ID', how='left',indicator = True)
mapping_df = mapping_df[mapping_df['Is Track Local'] == False]
mapping_df = mapping_df.merge(mapping_df[(mapping_df['_merge'] == 'both')][["Track External ID's","Track ID"]], on = ["Track External ID's"], how='left', suffixes=[""," Mapped"])
mapping_df = (
    mapping_df.merge(mapping_df[pd.isna(mapping_df['Track ID Mapped'])]
                     .groupby(["Track External ID's",])[["Track External ID's",'Track ID','Track Popularity']]
                     .apply(lambda x: x.sort_values(by = ['Track Popularity'], ascending=False).head(1))
                     .reset_index(drop = True)[["Track External ID's", "Track ID"]], on = ["Track External ID's"], how='left', suffixes=[""," Mapped New"])
)
mapping_df['Track ID Mapped'] = mapping_df.apply(lambda x: x['Track ID Mapped'] if not pd.isna(x['Track ID Mapped']) else x['Track ID Mapped New'], axis=1)
#mapping_df['Track ID Mapped'] = mapping_df.apply(lambda x: x['Track ID'] if pd.isna(x['Track ID Mapped']) else x['Track ID Mapped'], axis=1)
test_df = mapping_df.groupby(['Track ID', "Track External ID's"]).agg('count')
test_df = test_df[test_df['Track ID Mapped'] > 1].sort_values(by=['Track Name']).reset_index()
mapping_df = mapping_df[['Track ID', 'Track ID Mapped']]

if test_df.size == 0:
    del(test_df)
else:
    print('Number of Errors: ',test_df.size)

In [9]:
sorted = playlist_tracks.merge(playlists, how = 'left', left_on = 'Playlist ID', right_on = 'id')
sorted = (
    sorted[((sorted['description'] == "**") | (sorted['description'].apply(lambda x: x[0:3] == "** ")))]
    .merge(library, how = 'outer', on = 'Track ID', indicator = True)
    .merge(mapping_df, how = 'left', on = 'Track ID')
    .merge(tracks, how = 'left', on = 'Track ID')
    .merge(albums, how = 'left', on = 'Album ID')
)

In [10]:
playlists_to_add = pd.DataFrame(["New Music Friday UK",
                            "🆕🎧 New Alternative 40 Chart [Official] | Updated Weekly",
                            "Best tracks from 2023 albums ",
                            "Hot Tracks - 2023's best new singles",
                            "The Times Essential Tracks",
                            "The Needle Drop Weekly Track Roundup -  Fantano’s Best, Mehst and Worst New Tracks",
                            "Discover Weekly",
                            "Release Radar",
                            "R1 Weekly Playlist",
                            "BBC Radio 6 Music 2023",
                            "Indie Show with Jack Saunders",
                            "Radio 1's Future Sounds with Clara Amfo",
                            "Radio 1's Future Artists with Jack Saunders",
                            "Radio 1's Future Alternative",
                            "Radio 1's Chillest Show",
                            "Radio 1's Power Down Playlist with Sian Eleri",
                            "BBC Introducing on Radio 1",
                            "New Music Fix",
                            "Steve Lamacq",
                            ], columns = ["name"])

playlists_to_add = pd.merge(playlists_to_add,playlists, on = 'name', how = 'left')['id']
playlists_to_add = pd.merge(playlists_to_add,playlist_tracks, left_on = 'id', right_on = 'Playlist ID', how = 'left')

In [11]:
current_date_time = (datetime.today()).strftime('%Y-%m-%dT%H:%M:%SZ')

with open('./date.txt', 'r+') as f:
    last_date_time = f.read()
    f.seek(0)
    f.write(current_date_time)
    f.close()
    
print('Adding Tracks From: ', last_date_time, ' To: ', current_date_time)

tracks_to_add = playlists_to_add[playlists_to_add['Date Added']>=last_date_time][['Track ID']].drop_duplicates()
tracks_to_add['Playlist ID'] = '65jX0kBNpAEO7TcDQrkDai'
tracks_to_add['name'] = 'TBC'
tracks_to_add['Position Number'] = np.arange(len(tracks_to_add))
tracks_to_add['Date Added'] = date.today()
tracks_to_add['description'] = "*****"

del(f, playlists_to_add)

Adding Tracks From:  2023-09-10T23:34:43Z  To:  2023-09-15T11:39:11Z


In [12]:
temp = playlist_tracks.merge(playlists, how = 'left', left_on = 'Playlist ID', right_on = 'id')
temp = temp[((temp['description'] == "*****") | (temp['description'].apply(lambda x: x[0:6] == "***** ")))]
temp = temp.append(tracks_to_add)
temp = (
    temp.merge(library, how = 'left', on = 'Track ID', indicator = True)
    .merge(mapping_df, how = 'left', on = 'Track ID')
    .merge(tracks, how = 'left', on = 'Track ID')
)

custom_dict = {'To Listen To Albums': 0, 'To Listen To': 1, 'TBC': 2,'To Listen To Relax': 3} 

temp = temp[temp['name'] != 'Albums to Listen To']
temp.sort_values(by = 'name', key=lambda x: x.map(custom_dict), inplace=True)
temp = temp.groupby('Track ID Mapped').head(1)[['Track ID', 'Track ID Mapped', 'Playlist ID', 'Position Number', 'name', 'Track Name', '_merge']]
temp = temp[temp['name'] != 'To Listen To Albums']
temp = temp.merge(temp[['Track ID', '_merge']].rename(columns = {'Track ID': 'Track ID Mapped', '_merge': '_merge Mapped'}), how = 'left', on = 'Track ID Mapped')

del(custom_dict)

In [13]:
for playlist_id in tqdm(list(temp['Playlist ID'].drop_duplicates())):
    
    df = temp[temp['Playlist ID'] == playlist_id]
    df = df[((df['_merge'] != 'both') & (df['_merge Mapped'] != 'both'))]
    new_tracks = list(df['Track ID Mapped'])
    edit_playlist_tracks(playlist_id, new_tracks, order=False)
    
del(df, temp, new_tracks, playlist_id)

100%|██████████| 2/2 [02:03<00:00, 61.96s/it]


In [14]:
for playlist_id in tqdm(list(sorted['Playlist ID'].drop_duplicates())):
    
    df = sorted[sorted['Playlist ID'] == playlist_id][['Track ID', 'Track Name', 'Playlist ID', 'Album Name', 'Track Number', 'Disc Number','Album Release Date']]
    df.sort_values(by = ['Album Release Date', 'Album Name', 'Disc Number', 'Track Number'], ascending = [False, True, True, True], inplace=True)
    new_tracks = list(df['Track ID'])
    print(playlist_id)
    edit_playlist_tracks(playlist_id, new_tracks, order=True)
    
del(df, new_tracks, playlist_id)

  0%|          | 0/38 [00:00<?, ?it/s]

3j9PgqJK0oJFgk5caq828C


  3%|▎         | 1/38 [00:02<01:26,  2.35s/it]

30JEXgyxPC3vl06wlDovbZ


  5%|▌         | 2/38 [00:03<01:13,  2.04s/it]

...

 95%|█████████▍| 36/38 [04:01<00:07,  3.63s/it]

12qJcBTJvKgsuokKEQxm0n


 97%|█████████▋| 37/38 [04:02<00:03,  3.04s/it]

2M9B5ewM3fuqJq83RyF9y6


100%|██████████| 38/38 [04:05<00:00,  6.46s/it]


In [167]:
playlist_tracks.to_pickle('../data/spotify_playlist_tracks.pkl')
playlists.to_pickle('../data/spotify_playlists.pkl')
tracks.to_pickle('../data/spotify_tracks.pkl')
albums.to_pickle('../data/spotify_albums.pkl')
artists.to_pickle('../data/spotify_artists.pkl')
library.to_pickle('../data/spotify_library.pkl')
top_tracks.to_pickle('../data/spotify_top_tracks.pkl')

In [ ]:
###

In [15]:
def isrc_to_id(isrc):
    try:
        temp = spotify.search(q='isrc: ' + isrc, limit=1, type='track', market='GB')['tracks']['items'][0]
        return (temp['album']['name'],temp['album']['release_date'],temp['name'],temp['id'])
    except:
        return False
    
sorted[['New Album Name', 'Release Date', 'New Track Name', 'Latest ID']] = sorted['Track External ID\'s'].progress_apply(lambda x: pd.Series(isrc_to_id(x)))
sorted.to_pickle('../data/sorted.pkl')

In [16]:
sorted = pd.read_pickle('../data/sorted.pkl')

In [17]:
sorted_wrong = sorted[(sorted['Track ID'] != sorted['Latest ID']) | (sorted['Track Name'] != sorted['New Track Name'])][['Track Name', 'Track External ID\'s', 'Album Name', 'Album Release Date', 'Track ID', 'Track ID Mapped', 'Latest ID', 'Release Date','New Track Name', 'New Album Name']]
#sorted_wrong = sorted

sorted_wrong['New Track Name'].fillna(sorted_wrong['Track Name'], inplace=True)
sorted_wrong['New Album Name'].fillna(sorted_wrong['Album Name'], inplace=True)
sorted_wrong['Latest ID'].fillna(sorted_wrong['Track ID'], inplace=True)
sorted_wrong['Release Date'].fillna(sorted_wrong['Album Release Date'], inplace=True)
sorted_wrong = sorted_wrong.astype(str)

sorted_wrong['Partial Ratio'] = sorted_wrong.apply(lambda x: fuzz.partial_ratio(x['Track Name'], x['New Track Name']), axis=1)
sorted_wrong['Ratio'] = sorted_wrong.apply(lambda x: fuzz.ratio(x['Track Name'], x['New Track Name']), axis=1)
sorted_wrong['WRatio'] = sorted_wrong.apply(lambda x: fuzz.WRatio(x['Track Name'], x['New Track Name']), axis=1)

sorted_wrong['New Track Name'] = sorted_wrong.apply(lambda x: x['New Track Name'] if x['Partial Ratio']>70 else x['Track Name'], axis=1)
sorted_wrong['New Album Name'] = sorted_wrong.apply(lambda x: x['New Album Name'] if x['Partial Ratio']>70 else x['Album Name'], axis=1)
sorted_wrong['Latest ID'] = sorted_wrong.apply(lambda x: x['Latest ID'] if x['Partial Ratio']>70 else x['Track ID'], axis=1)
sorted_wrong['Release Date'] = sorted_wrong.apply(lambda x: x['Release Date'] if x['Partial Ratio']>70 else x['Album Release Date'], axis=1)

sorted_wrong['Partial Ratio'] = sorted_wrong.apply(lambda x: fuzz.partial_ratio(x['Track Name'], x['New Track Name']), axis=1)
sorted_wrong['Ratio'] = sorted_wrong.apply(lambda x: fuzz.ratio(x['Track Name'], x['New Track Name']), axis=1)
sorted_wrong['WRatio'] = sorted_wrong.apply(lambda x: fuzz.WRatio(x['Track Name'], x['New Track Name']), axis=1)

In [11]:
#make_playlist(title, description)
#edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

#edit_playlist_details(user=my_id, playlist_id = playlist_id, name = name', description = 'description')
#get_playlist(playlist_id, with_tracks = False)
#get_playlist(playlist_id = playlist_id)
#delete_playlist(playlist_id = playlist_id)